In [ ]:
"""
This script fills out the hypothetical_regular_form field in cards.csv
Each cell is for a different tense/person combination.

I do not write to cards.csv directly, but instead use to_clipboard, overwriting cards.csv may cause mojibake
"""

In [4]:
import pandas as pd

# Load the CSV
df = pd.read_csv('cards.csv')

REFL_PRON = {          # pronouns for reflexive forms
    '1st_singular':'me', '2nd_singular':'te', '3rd_singular':'se', '2nd_singular_polite':'se',
    '1st_plural':'nos',  '2nd_plural':'os',   '3rd_plural':'se',   '2nd_plural_polite':'se'
}

In [ ]:
"""
Add hypothetical gerundio forms to cards.csv 
"""

def get_gerundio(verb):

    # Handle reflexive verbs ending in 'se'
    is_reflexive = verb.endswith('se')
    base = verb[:-2] if is_reflexive else verb

    if base.endswith('ar'):
        form = base[:-2] + 'ando'
    elif base.endswith('er') or base.endswith('ir') or base.endswith('ír'):
        form = base[:-2] + 'iendo'
    else:
        form = ''
    if is_reflexive:
        form += 'se'


    return form

# Only update rows where tense is 'gerundio'
mask = df['tense'] == 'gerundio'
df.loc[mask, 'hypothetical_regular_form'] = df.loc[mask, 'verb'].apply(get_gerundio)

# copy the relevant columns to clipboard
df[df['tense']=='gerundio'][['card_id', 'hypothetical_regular_form']].to_clipboard(index=False, header=False)

In [ ]:
"""
Add hypothetical participio forms to cards.csv
"""

def get_participio(verb):

    if verb.endswith('ar'):
        form = verb[:-2] + 'ado'
    elif verb.endswith('er') or verb.endswith('ir') or verb.endswith('ír'):
        form = verb[:-2] + 'ido'
    else:
        form = ''

    return form

# Only update rows where tense is 'participio'
mask = df['tense'] == 'participio'
df.loc[mask, 'hypothetical_regular_form'] = df.loc[mask, 'verb'].apply(get_participio)

# copy the relevant columns to clipboard
df[df['tense']=='participio'][['card_id', 'hypothetical_regular_form']].to_clipboard(index=False, header=False)

In [ ]:
# --------------------------------------------
# Regular-verb ⊙ indicativo presente ⊙ helper
# --------------------------------------------


SUFFIXES = {
    # person            -ar      -er      -ir
    '1st_singular':     ('o',    'o',     'o'),
    '2nd_singular':     ('as',   'es',    'es'),
    '3rd_singular':     ('a',    'e',     'e'),
    '1st_plural':       ('amos', 'emos',  'imos'),
    '2nd_plural':       ('áis',  'éis',   'ís'),  
    '3rd_plural':       ('an',   'en',    'en')
}


def regular_present(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive  = verb.endswith('se')
    core_verb  = verb[:-2] if reflexive else verb            # strip “se” if present

    # Identify conjugation group
    group = core_verb[-2:]
    group = 'ir' if group == 'ír' else group                 # treat “ír” as –ir
    if group not in ('ar', 'er', 'ir'):
        return ''                                            # safety for odd entries

    stem   = core_verb[:-2]
    ending = SUFFIXES[person][('ar','er','ir').index(group)]
    form   = f"{stem}{ending}"

    if reflexive:
        form = f"{REFL_PRON[person]} {form}"

    return form.strip()

# -----------------------------------------------------
# Fill the column for all ⊙ indicativo presente ⊙ rows
# -----------------------------------------------------
mask = df['tense'] == '⊙ indicativo presente ⊙'
df.loc[mask, 'hypothetical_regular_form'] = df[mask].apply(regular_present, axis=1)

# -------------------------
# Result just as you need
# -------------------------
result_presente = df.loc[mask, ['card_id', 'hypothetical_regular_form']]
result_presente.to_clipboard(index=False, header=False)


In [ ]:
# ============================================================
# Regular-verb ⇠ indicativo imperfecto ⇠ helper
# ============================================================


IMPF_SUFFIXES = {
    # person_grammatical       -ar           -er          -ir
    '1st_singular':            ('aba',       'ía',        'ía'),
    '2nd_singular':            ('abas',      'ías',       'ías'),
    '3rd_singular':            ('aba',       'ía',        'ía'),
    '1st_plural':              ('ábamos',    'íamos',     'íamos'),
    '2nd_plural':              ('abais',     'íais',      'íais'),
    '3rd_plural':              ('aban',      'ían',       'ían'),
}

def regular_imperfect(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive   = verb.endswith('se')
    core_verb   = verb[:-2] if reflexive else verb        # strip “se” if present

    group = core_verb[-2:]
    group = 'ir' if group == 'ír' else group              # treat “ír” as –ir
    if group not in ('ar', 'er', 'ir'):
        return ''                                         # guard for odd data

    stem   = core_verb[:-2]
    ending = IMPF_SUFFIXES[person][('ar','er','ir').index(group)]
    form   = f"{stem}{ending}"

    if reflexive:
        form = f"{REFL_PRON[person]} {form}"

    return form.strip()

# ------------------------------------------------------------
# Fill the column for all ⇠ indicativo imperfecto ⇠ rows
# ------------------------------------------------------------
mask_impf = df['tense'] == '⇠ indicativo imperfecto ⇠'
df.loc[mask_impf, 'hypothetical_regular_form'] = df[mask_impf].apply(
    regular_imperfect, axis=1
)

# ------------------------------------------------------------
# Result dataframe (card_id + hypothetical_regular_form)
# ------------------------------------------------------------
result_impf = df.loc[mask_impf, ['card_id', 'hypothetical_regular_form']]
result_impf.to_clipboard(index=False, header=False)


In [26]:
# ============================================================
# Regular-verb ↧ indicativo pretérito ↧ helper
# ============================================================

PRET_SUFFIXES = {
    # person_grammatical       -ar           -er          -ir
    '1st_singular':            ('é',         'í',         'í'),
    '2nd_singular':            ('aste',      'iste',      'iste'),
    '3rd_singular':            ('ó',         'ió',        'ió'),
    '1st_plural':              ('amos',      'imos',      'imos'),
    '2nd_plural':              ('asteis',    'isteis',    'isteis'),
    '3rd_plural':              ('aron',      'ieron',     'ieron'),
}

def regular_preterite(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive   = verb.endswith('se')
    core_verb   = verb[:-2] if reflexive else verb

    group = core_verb[-2:]
    group = 'ir' if group == 'ír' else group
    if group not in ('ar', 'er', 'ir'):
        return ''

    stem   = core_verb[:-2]
    ending = PRET_SUFFIXES[person][('ar','er','ir').index(group)]
    form   = f"{stem}{ending}"

    if reflexive:
        form = f"{REFL_PRON[person]} {form}"

    return form.strip()

# ------------------------------------------------------------
# Fill the column for all ↧ indicativo pretérito ↧ rows
# ------------------------------------------------------------
mask_pret = df['tense'] == '↧ indicativo pretérito ↧'
df.loc[mask_pret, 'hypothetical_regular_form'] = df[mask_pret].apply(
    regular_preterite, axis=1
)

# ------------------------------------------------------------
# Result dataframe (card_id + hypothetical_regular_form)
# ------------------------------------------------------------
result_pret = df.loc[mask_pret, ['card_id', 'hypothetical_regular_form']]
result_pret.to_clipboard(index=False, header=False)


In [24]:
# ============================================================
# Regular-verb → indicativo futuro → helper
# ============================================================

FUTURO_SUFFIXES = {
    # person_grammatical       -ar/-er/-ir
    '1st_singular':    'é',
    '2nd_singular':    'ás',
    '3rd_singular':    'á',
    '1st_plural':      'emos',
    '2nd_plural':      'éis',
    '3rd_plural':      'án',
}

def regular_futuro(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive   = verb.endswith('se')
    core_verb   = verb[:-2] if reflexive else verb

    # For futuro, use the full infinitive (without 'se' if reflexive)
    stem = core_verb
    ending = FUTURO_SUFFIXES.get(person, '')
    form = f"{stem}{ending}"

    if reflexive:
        form = f"{REFL_PRON[person]} {form}"

    return form.strip()

# ------------------------------------------------------------
# Fill the column for all → indicativo futuro → rows
# ------------------------------------------------------------
mask_futuro = df['tense'] == '→ indicativo futuro →'
df.loc[mask_futuro, 'hypothetical_regular_form'] = df[mask_futuro].apply(
    regular_futuro, axis=1
)

# ------------------------------------------------------------
# Result dataframe (card_id + hypothetical_regular_form)
# ------------------------------------------------------------
result_futuro = df.loc[mask_futuro, ['card_id', 'hypothetical_regular_form']]
result_futuro.to_clipboard(index=False, header=False)


In [27]:
# ============================================================
# Regular-verb ◇ condicional ◇ helper
# ============================================================

CONDICIONAL_SUFFIXES = {
    # person_grammatical       -ar/-er/-ir
    '1st_singular':    'ía',
    '2nd_singular':    'ías',
    '3rd_singular':    'ía',
    '1st_plural':      'íamos',
    '2nd_plural':      'íais',
    '3rd_plural':      'ían',
}

def regular_condicional(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive   = verb.endswith('se')
    core_verb   = verb[:-2] if reflexive else verb

    # For condicional, use the full infinitive (without 'se' if reflexive)
    stem = core_verb
    ending = CONDICIONAL_SUFFIXES.get(person, '')
    form = f"{stem}{ending}"

    if reflexive:
        form = f"{REFL_PRON[person]} {form}"

    return form.strip()

# ------------------------------------------------------------
# Fill the column for all ◇ condicional ◇ rows
# ------------------------------------------------------------
mask_condicional = df['tense'] == '◇ condicional ◇'
df.loc[mask_condicional, 'hypothetical_regular_form'] = df[mask_condicional].apply(
    regular_condicional, axis=1
)

# ------------------------------------------------------------
# Result dataframe (card_id + hypothetical_regular_form)
# ------------------------------------------------------------
result_condicional = df.loc[mask_condicional, ['card_id', 'hypothetical_regular_form']]
result_condicional.to_clipboard(index=False, header=False)


In [28]:
# --------------------------------------------
# Regular-verb ∿ subjuntivo presente ∿ helper
# --------------------------------------------

SUBJ_PRESENT_SUFFIXES = {
    # person            -ar      -er      -ir
    '1st_singular':     ('e',    'a',     'a'),
    '2nd_singular':     ('es',   'as',    'as'),
    '3rd_singular':     ('e',    'a',     'a'),
    '1st_plural':       ('emos', 'amos',  'amos'),
    '2nd_plural':       ('éis',  'áis',   'áis'),
    '3rd_plural':       ('en',   'an',    'an')
}

def regular_subj_present(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive  = verb.endswith('se')
    core_verb  = verb[:-2] if reflexive else verb

    group = core_verb[-2:]
    group = 'ir' if group == 'ír' else group
    if group not in ('ar', 'er', 'ir'):
        return ''

    stem   = core_verb[:-2]
    ending = SUBJ_PRESENT_SUFFIXES[person][('ar','er','ir').index(group)]
    form   = f"{stem}{ending}"

    if reflexive:
        form = f"{REFL_PRON[person]} {form}"

    return form.strip()

# -----------------------------------------------------
# Fill the column for all ∿ subjuntivo presente ∿ rows
# -----------------------------------------------------
mask_subj = df['tense'] == '∿ subjuntivo presente ∿'
df.loc[mask_subj, 'hypothetical_regular_form'] = df[mask_subj].apply(regular_subj_present, axis=1)

# -------------------------
# Result just as you need
# -------------------------
result_subj_presente = df.loc[mask_subj, ['card_id', 'hypothetical_regular_form']]
result_subj_presente.to_clipboard(index=False, header=False)


In [29]:
# --------------------------------------------
# Regular-verb ↫ subjuntivo imperfecto ↫ helper
# --------------------------------------------

SUBJ_IMPF_SUFFIXES = {
    # person            -ar        -er/-ir
    '1st_singular':     ('ara',    'iera'),
    '2nd_singular':     ('aras',   'ieras'),
    '3rd_singular':     ('ara',    'iera'),
    '1st_plural':       ('áramos', 'iéramos'),
    '2nd_plural':       ('arais',  'ierais'),
    '3rd_plural':       ('aran',   'ieran')
}

def regular_subj_imperfecto(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive = verb.endswith('se')
    core_verb = verb[:-2] if reflexive else verb

    group = core_verb[-2:]
    group = 'ir' if group == 'ír' else group
    if group not in ('ar', 'er', 'ir'):
        return ''

    stem = core_verb[:-2]
    if group == 'ar':
        ending = SUBJ_IMPF_SUFFIXES[person][0]
    else:
        ending = SUBJ_IMPF_SUFFIXES[person][1]
    form = f"{stem}{ending}"

    if reflexive:
        form = f"{REFL_PRON[person]} {form}"

    return form.strip()

# -----------------------------------------------------
# Fill the column for all ↫ subjuntivo imperfecto ↫ rows
# -----------------------------------------------------
mask_subj_impf = df['tense'] == '↫ subjuntivo imperfecto ↫'
df.loc[mask_subj_impf, 'hypothetical_regular_form'] = df[mask_subj_impf].apply(regular_subj_imperfecto, axis=1)

# -------------------------
# Result just as you need
# -------------------------
result_subj_imperfecto = df.loc[mask_subj_impf, ['card_id', 'hypothetical_regular_form']]
result_subj_imperfecto.to_clipboard(index=False, header=False)


In [5]:
# --------------------------------------------
# Regular-verb ¡ imperativo afirmativo ! helper
# --------------------------------------------

IMPERATIVO_AFIRMATIVO_SUFFIXES = {
    # person                    -ar      -er      -ir
    '2nd_singular':            ('a',     'e',     'e'),     # tú
    '2nd_singular_polite':     ('e',     'a',     'a'),     # usted
    '1st_plural':              ('emos',  'amos',  'amos'),  # nosotros
    '2nd_plural_polite':       ('en',    'an',    'an'),    # ustedes
}

def regular_imperativo_afirmativo(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive = verb.endswith('se')
    core_verb = verb[:-2] if reflexive else verb

    group = core_verb[-2:]
    group = 'ir' if group == 'ír' else group
    if group not in ('ar', 'er', 'ir'):
        return ''

    stem = core_verb[:-2]
    ending = IMPERATIVO_AFIRMATIVO_SUFFIXES.get(person, ('', '', ''))[('ar', 'er', 'ir').index(group)]
    form = f"{stem}{ending}"

    if reflexive:
        # For imperativo afirmativo, reflexive pronoun is attached at the end (no space)
        form = f"{form}{REFL_PRON[person]}"

    return form.strip()

# ------------------------------------------------------------
# Fill the column for all ¡ imperativo afirmativo ! rows
# ------------------------------------------------------------
mask_imperativo_af = df['tense'] == '¡ imperativo afirmativo !'
df.loc[mask_imperativo_af, 'hypothetical_regular_form'] = df[mask_imperativo_af].apply(
    regular_imperativo_afirmativo, axis=1
)

# ------------------------------------------------------------
# Result dataframe (card_id + hypothetical_regular_form)
# ------------------------------------------------------------
result_imperativo_af = df.loc[mask_imperativo_af, ['card_id', 'hypothetical_regular_form']]
result_imperativo_af.to_clipboard(index=False, header=False)


In [6]:
# --------------------------------------------
# Regular-verb ¡ imperativo negativo ! helper
# --------------------------------------------

IMPERATIVO_NEGATIVO_SUFFIXES = {
    # person                    -ar      -er      -ir
    '2nd_singular':            ('es',    'as',    'as'),     # tú
    '2nd_singular_polite':     ('e',     'a',     'a'),      # usted (same as afirmativo)
    '1st_plural':              ('emos',  'amos',  'amos'),   # nosotros (same as afirmativo)
    '2nd_plural_polite':       ('en',    'an',    'an'),     # ustedes (same as afirmativo)
}

def regular_imperativo_negativo(row):
    verb, person = row['verb'], row['person_grammatical']
    reflexive = verb.endswith('se')
    core_verb = verb[:-2] if reflexive else verb

    group = core_verb[-2:]
    group = 'ir' if group == 'ír' else group
    if group not in ('ar', 'er', 'ir'):
        return ''

    stem = core_verb[:-2]
    ending = IMPERATIVO_NEGATIVO_SUFFIXES.get(person, ('', '', ''))[('ar', 'er', 'ir').index(group)]
    form = f"no {stem}{ending}"

    if reflexive:
        # For imperativo negativo, reflexive pronoun goes before the verb, after "no"
        form = f"no {REFL_PRON[person]} {stem}{ending}"

    return form.strip()

# ------------------------------------------------------------
# Fill the column for all ¡ imperativo negativo ! rows
# ------------------------------------------------------------
mask_imperativo_neg = df['tense'] == '¡ imperativo negativo !'
df.loc[mask_imperativo_neg, 'hypothetical_regular_form'] = df[mask_imperativo_neg].apply(
    regular_imperativo_negativo, axis=1
)

# ------------------------------------------------------------
# Result dataframe (card_id + hypothetical_regular_form)
# ------------------------------------------------------------
result_imperativo_neg = df.loc[mask_imperativo_neg, ['card_id', 'hypothetical_regular_form']]
result_imperativo_neg.to_clipboard(index=False, header=False)
